In [91]:
import os
from pathlib import Path
import IPython.display as ipd
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
from IPython import display
import time
from datetime import datetime, timedelta
import tensorflow as tf
import seaborn as sns
import subprocess
%matplotlib inline

tf.logging.set_verbosity(tf.logging.ERROR)

# NOTES
NOTES = "28x28"

# VARS
target_label = "Label"
id_label = "fname"
VERBOSE = True
DISPLAY = True
TEST = False
TPU = False
RESIZE = True
INPUT_WIDTH = 128
INPUT_HEIGHT = 128
TARGET_WIDTH = 28 if RESIZE else INPUT_WIDTH
TARGET_HEIGHT = 28 if RESIZE else INPUT_HEIGHT
DECAY_RATE = 0.9
IMG_CHANNELS = 3
DROPOUT = 0.4
TYPE = "CNN"
DEFAULT_BS = 128 # default batch size
UNK_DROP_RATE = 1.0 # drop 100% of unknown categories
OUTLIER_PERCENTAGE = 0.1

FNAME = "subvocal_12_19"
CATEGORY = ["no_voice"]
LABELS = ["lights-on", "turn-off", "---"]
CHANNELS = [1, 2, 3, 4]
NUMS = ''.join([str(x) for x in CHANNELS])
MONTHS = [12]
DAYS = [19]

if TEST:
    LEARNING_STEPS = 100
    SPP = 4
    LEARNING_RATE = .05
    BATCH_SIZE = 32
    VERBOSITY = 1000
    TEST_SIZE = 1000
    SHUFFLE_SIZE = 64
else:
    LEARNING_STEPS = 5000
    SPP = 200
    LEARNING_RATE = .025
    BATCH_SIZE = 64
    VERBOSITY = 1000
    SHUFFLE_SIZE = 256

def curr_time():
    return datetime.now() - timedelta(hours=7) # offset from UTC to PST

ROOT = os.getcwd() + "/"
if CATEGORY[0] == "no_voice":
    RUN_ROOT = ROOT+"NONVOCAL_RUNS_YN_{:02}_{:02}/".format(MONTHS[0], DAYS[0])
else:
    RUN_ROOT = ROOT+"VOCAL_RUNS_YN_{:02}_{:02}/".format(MONTHS[0], DAYS[0])
RUN_ROOT_LOG = RUN_ROOT+"logs/"

# PATHS
paths = {
    "Training":ROOT+"train_csv/" + FNAME + ".csv",
    "Model":RUN_ROOT+"models/NONVOCAL_RUNS_YN_12_19_subvocal_4chunks_74%/model_dir_1234",# model_dir_{}/.format(NUMS),
    "Logs":RUN_ROOT_LOG+"{}_{}/".format(NUMS, datetime.strftime(curr_time(), "%b%d%Y_%H%M%S"))
}
paths["Log"] = paths["Logs"] + "log.txt"
if not os.path.isdir(RUN_ROOT):
    os.mkdir(RUN_ROOT)
if not os.path.isdir(RUN_ROOT_LOG):
    os.mkdir(RUN_ROOT_LOG)
if not os.path.isdir(paths["Logs"]):
    os.mkdir(paths["Logs"])
    
print(paths['Model'])

/Users/kyy/cerebro_train/NONVOCAL_RUNS_YN_12_19/models/NONVOCAL_RUNS_YN_12_19_subvocal_4chunks_74%/model_dir_1234


In [92]:
def make_header(s):
    return ("#" * 42) + ("\n{:^42}\n".format(s)) + ("#" * 42)
    
def print_and_log(s):
    with open(paths["Log"], 'a') as log:
        log.write(str(s))
        log.write("\n")
    print(s)
        
def print_and_log_header(s):
    h = make_header(str(s))
    with open(paths["Log"], 'a') as log:
        log.write(h)
        log.write("\n")
    print(h)

In [93]:
def sec_to_str(secs):
    ms = secs - int(secs)
    days = int(secs // (24 * 3600))
    hours = int((secs % ((24 * 3600))) // 3600)
    minutes = int((secs % 3600) // 60)
    seconds = int(secs % 60)
    return "{:02}:{:02}:{:02}:{:02}.{}".format(days, hours, minutes, seconds, "{:.3}".format(ms)[2:])

def timer(f, *args):
    print_and_log("Start: {}".format(curr_time()))
    start = time.time()
    result = f(*args)
    end = time.time()
    print_and_log("End: {}".format(curr_time()))
    print_and_log("Finished in {}".format(sec_to_str(end - start)))
    return result

def preprocess(samples, sample_rate):
    padded = np.zeros(sample_rate)
    samples = samples[:sample_rate]
    padded[:samples.shape[0]] = samples
    return padded

def select_labels(df, allowed):
    return df[df['Label'].isin(allowed)]
    
def select_categories(df, allowed):
    return df[df['Category'].isin(allowed)]

def select_channels(df, allowed):
    labels = []
    for i in range(1, 5):
        if i not in allowed:
            labels.append("Path{}".format(i))
    return df.drop(labels, axis=1)

def select_days(df, allowed):
    return df[df['Day'].isin(allowed)]

def select_months(df, allowed):
    return df[df['Month'].isin(allowed)]

def select_sets(df, allowed):
    return df[df['Set'].isin(allowed)]

def remove_voice(df):
    return df.drop(["Path4"], axis=1)

def str_to_l(x):
    return [int(n) for n in x if n <= '9' and n >= '0']

In [94]:
count = 0
def _parse_function(label, *filenames):
    global count
    count += 1
    if count % VERBOSITY == 0:
        print_and_log("\tProcessed {}th image".format(count))
    expected_shape = tf.constant([1, INPUT_HEIGHT, INPUT_WIDTH, IMG_CHANNELS])
    image = None
    for filename in filenames:
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_image(image_string, channels=IMG_CHANNELS)
        image_decoded = tf.image.convert_image_dtype(image_decoded, tf.float32)
        image_decoded = tf.reshape(image_decoded, expected_shape)
        image_decoded = tf.image.rgb_to_grayscale(image_decoded)
        if RESIZE:
            image_decoded = tf.image.resize_bicubic(image_decoded, [TARGET_HEIGHT, TARGET_WIDTH])
        if image is not None:
            image = tf.concat([image, image_decoded], 3)
        else:
            image = image_decoded
    return image, label

In [95]:
def model_fn(features, labels, mode):
    input_layer = tf.reshape(features, [-1, TARGET_HEIGHT, TARGET_WIDTH, len(CHANNELS)])
    pool = input_layer

    for num_filters in [32, 64]:
        conv = tf.layers.conv2d(
            inputs=pool,
            filters=num_filters,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)
        pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool = tf.layers.flatten(pool)
    dense = tf.layers.dense(inputs=pool, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=DROPOUT, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=num_labels)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if not TPU:
        tf.summary.histogram("predictions", predictions["probabilities"])
        tf.summary.histogram("classes", predictions["classes"])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    learning_rate = tf.train.exponential_decay(LEARNING_RATE, tf.train.get_global_step(), SPP, DECAY_RATE, staircase=True)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    predictions["loss"] = loss
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        if TPU:
            optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [96]:
def create_training_input_fn(dataset, batch_size, num_epochs=None):
    def _input_fn(num_epochs=None, shuffle=True):
        ds = dataset.batch(batch_size).repeat(num_epochs)
        if shuffle:
            ds = ds.shuffle(SHUFFLE_SIZE)
        feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
        return feature_batch, label_batch
    return _input_fn

def create_predict_input_fn(dataset, batch_size):
    def _input_fn():
        ds = dataset.batch(batch_size)
        feature_batch, label_batch = ds.make_one_shot_iterator().get_next()
        return feature_batch, label_batch
    return _input_fn

In [97]:
with open(paths["Log"], 'w') as log:
    log.write(make_header("Starting Script\n"))

In [98]:
# Create variables for the paths
train_csv = paths["Training"]

# Store the labels to train
all_labels = LABELS
labels = LABELS
num_labels = len(labels)
labels = {x[1]:x[0] for x in enumerate(labels)}
reverse_lookup = {labels[k]:k for k in labels}

In [99]:
# Make the training data
print_and_log_header("MAKING TRAINING DATA")
train_data = pd.read_csv(train_csv)

# Filter the training data
train_data = select_categories(train_data, CATEGORY)
train_data = select_channels(train_data, CHANNELS)
train_data = select_labels(train_data, LABELS)
train_data = select_months(train_data, MONTHS)
train_data = select_days(train_data, DAYS)
# train_data = remove_voice(train_data)

train_data = train_data.sample(frac=1).reset_index(drop=True)
tdcopy = pd.DataFrame(train_data)
train_data["Label"] = train_data["Label"].map(labels)

if VERBOSE:
    print_and_log_header("TRAIN DATA")
    print_and_log(train_data.describe())
    print_and_log(train_data.head(10))

##########################################
           MAKING TRAINING DATA           
##########################################
##########################################
                TRAIN DATA                
##########################################
          Day   Month        Label  SequenceNumber
count  5312.0  5312.0  5312.000000     5312.000000
mean     19.0    12.0     1.000000      885.834337
std       0.0     0.0     0.816881      511.197214
min      19.0    12.0     0.000000        1.000000
25%      19.0    12.0     0.000000      443.000000
50%      19.0    12.0     1.000000      886.000000
75%      19.0    12.0     2.000000     1328.250000
max      19.0    12.0     2.000000     1772.000000
   Category  Day  Month  Label  SequenceNumber         Set  \
0  no_voice   19     12      0             551    Training   
1  no_voice   19     12      2            1037    Training   
2  no_voice   19     12      1             908  Validation   
3  no_voice   19     12      1     

In [100]:
# # Separate Labels
train_labels = train_data.pop(target_label)
img_paths = ["Path{}".format(channel) for channel in CHANNELS]
train_data = train_data[img_paths]

In [101]:
# Vectors of filenames.
t_f, v_f, s_f = [], [], []
for i in range(1, 1 + len(CHANNELS)):
    channel = CHANNELS[i-1]
    l = "Path{}".format(channel)
    t_f.append(tf.constant(train_data[l]))

# `labels[i]` is the label for the image in `filenames[i]
# Vectors of labels
train_labels = tf.constant(train_labels)

# Make datasets from filenames and labels
train_data = tf.data.Dataset.from_tensor_slices((train_labels, *t_f))
print_and_log_header("Parsing Training Data")
train_data = timer(lambda: train_data.map(_parse_function))
print_and_log("\nDone!")

##########################################
          Parsing Training Data           
##########################################
Start: 2018-12-19 12:23:59.907875
End: 2018-12-19 12:24:00.149801
Finished in 00:00:00:00.242

Done!


In [102]:
print_and_log_header("TRAINING")
print_and_log(train_data)
print_and_log(type(train_data))

##########################################
                 TRAINING                 
##########################################
<MapDataset shapes: ((1, 28, 28, 4), ()), types: (tf.float32, tf.int64)>
<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [103]:
# Create the Estimator
classifier = tf.estimator.Estimator(model_fn=model_fn, model_dir=paths["Model"])

In [104]:
# Create the input functions.
training_eval_input_fn = create_predict_input_fn(train_data, DEFAULT_BS)

In [105]:
# Create predicitons and remove 10% lowest confidence rows
results = [x for x in classifier.predict(input_fn=training_eval_input_fn)]
classes = [x["classes"] for x in results]
probs = [x["probabilities"] for x in results]
probs = pd.Series(probs)
probs = probs.apply(lambda x: max(x))
tdf = pd.DataFrame({"Prediction":classes, "Probability":probs})
num_items = tdf.shape[0]
for k in tdcopy:
    tdf[k] = tdcopy[k]
outliers = tdf.nsmallest(int(num_items * OUTLIER_PERCENTAGE), "Probability")
keepers = tdf.append(outliers, ignore_index=True).drop_duplicates(["Day", "Month", "Label", "SequenceNumber"], keep=False).reset_index(drop=True)
print(tdf)
outliers = outliers.reset_index(drop=True)
keepers["Label"] = keepers["Label"].apply(lambda x: reverse_lookup[x])
outliers["Label"] = outliers["Label"].apply(lambda x: reverse_lookup[x])
if DISPLAY:
    print_and_log_header("OUTLIERS")
    display.display(outliers.describe())
    print_and_log_header("KEEPERS")
    display.display(keepers.describe())

      Prediction  Probability  Category  Day  Month  Label  SequenceNumber  \
0              0     0.357808  no_voice   19     12      0             551   
1              0     0.353428  no_voice   19     12      2            1037   
2              0     0.351302  no_voice   19     12      1             908   
3              0     0.347253  no_voice   19     12      1            1214   
4              0     0.355327  no_voice   19     12      0             239   
5              0     0.350945  no_voice   19     12      0             197   
6              0     0.353703  no_voice   19     12      2            1321   
7              0     0.351543  no_voice   19     12      1             243   
8              0     0.354090  no_voice   19     12      1             425   
9              0     0.348431  no_voice   19     12      2             244   
10             0     0.347828  no_voice   19     12      1            1198   
11             0     0.354940  no_voice   19     12      0      

,Prediction,Probability,Day,Month,SequenceNumber
count,531.000000,531.000000,531.0,531.0,531.000000
mean,0.625235,0.340449,19.0,12.0,886.403013
std,0.927994,0.001360,0.0,0.0,459.438529
min,0.000000,0.333460,19.0,12.0,4.000000
25%,0.000000,0.339762,19.0,12.0,579.500000
50%,0.000000,0.340702,19.0,12.0,871.000000
75%,2.000000,0.341359,19.0,12.0,1210.500000
max,2.000000,0.342191,19.0,12.0,1772.000000


##########################################
                 KEEPERS                  
##########################################


,Prediction,Probability,Day,Month,SequenceNumber
count,4781.000000,4781.000000,4781.0,4781.0,4781.000000
mean,0.011713,0.349090,19.0,12.0,885.771178
std,0.152623,0.003591,0.0,0.0,516.669756
min,0.000000,0.342200,19.0,12.0,1.000000
25%,0.000000,0.346322,19.0,12.0,433.000000
50%,0.000000,0.348965,19.0,12.0,889.000000
75%,0.000000,0.351798,19.0,12.0,1343.000000
max,2.000000,0.373470,19.0,12.0,1772.000000


In [106]:
# Drop unnecessary columns
keepers.drop(["Prediction", "Probability"], axis=1, inplace=True)
outliers.drop(["Prediction", "Probability"], axis=1, inplace=True)

In [107]:
s = outliers["Set"]
tra = sum(s.apply(lambda x: 1 if x == "Training" else 0))
val = sum(s.apply(lambda x: 1 if x == "Validation" else 0))
tst = sum(s.apply(lambda x: 1 if x == "Testing" else 0))
print("OUTLIERS\n\tTraining:\t{}\n\tValidation:\t{}\n\tTesting:\t{}".format(tra, val, tst))

OUTLIERS
	Training:	431
	Validation:	58
	Testing:	42


In [108]:
KEEPERS_NAME = "keepers_{:02}_{:02}_{}.csv".format(MONTHS[0], DAYS[0], NUMS)
OUTLIERS_NAME = "outliers_{:02}_{:02}_{}.csv".format(MONTHS[0], DAYS[0], NUMS)
keepers.to_csv(KEEPERS_NAME)
print("Saved KEEPERS to '{}'".format(KEEPERS_NAME))
outliers.to_csv(OUTLIERS_NAME)
print("Saved OUTLIERS to '{}'".format(OUTLIERS_NAME))

Saved KEEPERS to 'keepers_12_19_1234.csv'
Saved OUTLIERS to 'outliers_12_19_1234.csv'
